In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

2023-09-23 18:33:01.911442: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 18:33:02.814987: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-09-23 18:33:02.815286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [2]:
dataset_dir = 'Dataset'

In [3]:
img_height, img_width = 150, 150
batch_size = 32

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to [0, 1]
    validation_split=0.2,      # Split dataset into training (80%) and validation (20%)
    rotation_range=20,         # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,     # Randomly shift the width of images
    height_shift_range=0.2,    # Randomly shift the height of images
    horizontal_flip=True      # Randomly flip images horizontally
)

In [5]:
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Three or more classes
    subset='training'           # Specify 'training' for the training dataset
)

Found 1263 images belonging to 3 classes.


In [6]:
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Three or more classes
    subset='validation'         # Specify 'validation' for the validation dataset
)

Found 315 images belonging to 3 classes.


In [7]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(3, activation='softmax')  # Three classes
])

2023-09-23 18:33:04.336702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-23 18:33:04.364289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-23 18:33:04.364592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-23 18:33:04.365577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
epochs = 50  # Adjust the number of epochs as needed
model.fit(train_generator, validation_data=validation_generator, epochs=epochs)

Epoch 1/50


2023-09-23 18:33:06.130475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-09-23 18:33:06.476344: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-23 18:33:06.801565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-23 18:33:06.832701: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f746c03aef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-23 18:33:06.832730: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-09-23 18:33:06.837136: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-23 18:

40/40 [==============================] - 13s 253ms/step - loss: 6.6922 - accuracy: 0.5170 - val_loss: 2.3334 - val_accuracy: 0.2000
Epoch 2/50
40/40 [==============================] - 9s 213ms/step - loss: 3.6459 - accuracy: 0.5764 - val_loss: 2.3357 - val_accuracy: 0.2095
Epoch 3/50
40/40 [==============================] - 8s 211ms/step - loss: 2.1614 - accuracy: 0.5914 - val_loss: 3.7348 - val_accuracy: 0.1841
Epoch 4/50
40/40 [==============================] - 8s 213ms/step - loss: 2.3379 - accuracy: 0.6017 - val_loss: 1.7306 - val_accuracy: 0.2254
Epoch 5/50
40/40 [==============================] - 8s 213ms/step - loss: 1.8160 - accuracy: 0.5978 - val_loss: 1.8004 - val_accuracy: 0.2730
Epoch 6/50
40/40 [==============================] - 9s 215ms/step - loss: 1.7761 - accuracy: 0.6136 - val_loss: 1.9418 - val_accuracy: 0.2571
Epoch 7/50
40/40 [==============================] - 8s 212ms/step - loss: 1.5065 - accuracy: 0.6334 - val_loss: 3.4564 - val_accuracy: 0.2413
Epoch 8/50
40/40

In [12]:
validation_preds = model.predict(validation_generator)
predicted_classes = np.argmax(validation_preds, axis=1)
true_classes = validation_generator.classes

10/10 [==============================] - 2s 172ms/step


In [13]:
accuracy = accuracy_score(true_classes, predicted_classes)
f1_score = classification_report(true_classes, predicted_classes, target_names=train_generator.class_indices.keys())
print(f"Accuracy: {accuracy}")
print(f"F1 Score:\n{f1_score}")

Accuracy: 0.49206349206349204
F1 Score:
              precision    recall  f1-score   support

      benign       0.57      0.76      0.65       178
   malignant       0.26      0.13      0.17        84
      normal       0.24      0.15      0.19        53

    accuracy                           0.49       315
   macro avg       0.36      0.35      0.34       315
weighted avg       0.43      0.49      0.45       315

